In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan as ld
import scipy.misc
import numpy as np
from tqdm import tqdm

In [3]:
for fname in tqdm(ld.listdir("data/pets/images", ".jpg")):
    try:
        dir_name = fname.split("/")[-1].split("_")[0]
        if not os.path.isdir(dir_name):
            ld.mkdir("data/pets/imgs/" + dir_name)
        img = scipy.misc.imread(fname)
        fname_new = os.path.join("data/pets/imgs/", dir_name, fname.split("/")[-1])
        scipy.misc.imsave(fname_new, img)
    except:
        print "ERROR WITH", fname

 39%|███▊      | 2851/7390 [01:32<01:22, 55.26it/s]

ERROR WITH data/pets/images/Egyptian_Mau_177.jpg
ERROR WITH

 61%|██████    | 4518/7390 [02:53<02:35, 18.44it/s]

 data/pets/images/Egyptian_Mau_14.jpg
ERROR WITH

 77%|███████▋  | 5668/7390 [03:51<01:15, 22.89it/s]

 data/pets/images/Abyssinian_5.jpg
ERROR WITH

100%|██████████| 7390/7390 [05:11<00:00, 23.75it/s]

 data/pets/images/Egyptian_Mau_186.jpg


# delete trouble makers

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [2]:
import ld_gan
import ld_gan.utils.utils as ld
import scipy.misc
import numpy as np
from tqdm import tqdm

In [3]:
for p1 in tqdm(ld.listdir('data/pets/imgs/')):
    for p2 in ld.listdir(p1):
        try:
            scipy.misc.imread(p2)
        except:
            print "remove", p2
            os.remove(p2)

 14%|█▍        | 5/35 [00:04<00:31,  1.05s/it]

remove data/pets/imgs/Egyptian/Egyptian_Mau_177.jpg
remove data/pets/imgs/Egyptian/Egyptian_Mau_14.jpg
remove

 34%|███▍      | 12/35 [00:31<01:25,  3.73s/it]

 data/pets/imgs/Egyptian/Egyptian_Mau_186.jpg
remove

100%|██████████| 35/35 [01:36<00:00,  2.40s/it]

 data/pets/imgs/Abyssinian/Abyssinian_5.jpg


# set bounding boxes

In [2]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")

%matplotlib notebook
import matplotlib.pylab as plt
# pylab.rcParams['figure.figsize'] = (10, 5)

In [3]:
import ld_gan
import ld_gan.utils.utils as ld
import scipy.misc
import numpy as np
from tqdm import tqdm

In [4]:
def get_bb_for_fname(fname_img):
    fname_xml = "data/pets/annotations/xmls/" + fname_img.split("/")[-1][:-4] + ".xml"
    
    with open(fname_xml, "r") as f:
        content = f.read()
        
    xmin = int(content.split("<xmin>")[1].split("</xmin>")[0])
    xmax = int(content.split("<xmax>")[1].split("</xmax>")[0])
    ymin = int(content.split("<ymin>")[1].split("</ymin>")[0])
    ymax = int(content.split("<ymax>")[1].split("</ymax>")[0])
    
    return xmin, xmax, ymin, ymax

In [11]:
def get_cropped_img_for_fname(fname_img):
    xmin, xmax, ymin, ymax = get_bb_for_fname(fname_img)
    img = scipy.misc.imread(fname_img)
    img = img[ymin:ymax, xmin:xmax]
    return img

In [12]:
fnames = ld.listdir("data/pets/imgs/", file_type=".jpg", recrusive=True)

In [13]:
for fname in tqdm(fnames):
    try:
        img = get_cropped_img_for_fname(fname)
        parts = fname.split("/")
        parts[2] = "cropped" # HERE
        path = os.path.join(*parts[:-1])
        try:
            ld.mkdir(path)
        except:
            pass
        fname = os.path.join(*parts)
        scipy.misc.imsave(fname, img)
    except:
        pass

100%|██████████| 7386/7386 [01:34<00:00, 52.26it/s]


# load

In [14]:
X, Y = ld_gan.data_proc.data_loader.load_data("data/pets/cropped/")

100%|██████████| 35/35 [00:26<00:00,  1.12s/it]

load data from 'data/pets/cropped/'


# look at shape

In [16]:
shape = []
for x in X:
    shape.append(x.shape[0])

In [17]:
shape = np.array(shape)

In [18]:
shape.min(), shape.max(), shape.mean()

(22, 1535, 168.3041237113402)

# trian / test

In [95]:
fname_train = "data/pets/annotations/trainval.txt"
with open(fname_train, "r") as f:
    content = f.read()
fnames_train = content.split("\n")

fname_test = "data/pets/annotations/test.txt"
with open(fname_test, "r") as f:
    content = f.read()
fnames_test = content.split("\n")

In [96]:
len(fnames_test)

3670

In [97]:
len(fnames_train)

3681

In [99]:
fnames_train = [f.split(" ")[0] for f in fnames_train]

In [100]:
fnames_test = [f.split(" ")[0] for f in fnames_test]

In [101]:
fnames_train[0] in fnames_test

False